## DeepWideNet Architecture

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input, Dropout, concatenate
from tensorflow.keras.layers import AveragePooling2D, BatchNormalization, Add, Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import glorot_uniform, constant
from tensorflow.keras.activations import selu, softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import plot_model

In [2]:
def wide_block(x, filters, name):
    f1, f2, f3 = filters
    prev_layer = x

    # wide shortcut
    conv_1x1_shortcut = Conv2D(filters=f1, kernel_size=(1, 1), activation=selu, padding='same')(prev_layer)
    conv_1x1_shortcut = BatchNormalization(axis=3)(conv_1x1_shortcut)

    # sub-deep path
    conv_5x5 = Conv2D(filters=f2, kernel_size=(5, 5), strides=(2, 2), activation=selu, padding='same')(prev_layer)
    conv_5x5 = BatchNormalization(axis=3)(conv_5x5)
    conv_3x3 = Conv2D(filters=f3, kernel_size=(3, 3), strides=(1, 1), activation=selu, padding='same')(conv_5x5)
    conv_3x3 = BatchNormalization(axis=3)(conv_3x3)

    # concatenation block 1
    sub_deep_block = concatenate([prev_layer, conv_3x3], axis=3)
    sub_deep_block = BatchNormalization(axis=3)(sub_deep_block)

    # concatenation block 2
    final_concat = concatenate([conv_1x1_shortcut, sub_deep_block], axis=3, name=name)

    return final_concat